In [14]:
%run "../../../setup.ipynb"
if CLIENT == None:
    print("Failed to create CLIENT.")

Setup Complete. Use CLIENT to access Weaviate.


In [15]:
from langchain.chat_models import AzureChatOpenAI
from langchain.vectorstores import Weaviate
from typing import List
import os
import weaviate

def openai_chat_client():
  deployment = os.getenv("AZURE_OPENAI_LLM_DEPLOYMENT_ID")
  key = os.getenv("AZURE_OPENAI_API_KEY")
  resource = os.getenv("AZURE_OPENAI_RESOURCE_NAME")

  return AzureChatOpenAI(deployment_name=deployment, 
                         openai_api_key=key, 
                         openai_api_base=f"https://{resource}.openai.azure.com/",
                         openai_api_version="2023-03-15-preview")


def weaviate_vector_store(index_name: str, text_key: str, attributes: List[str] = []):
  weaviate_url = os.environ['WEAVIATE_URL']
  weaviate_api_key = os.environ['WEAVIATE_API_KEY']
  openai_api_key = os.environ['AZURE_OPENAI_API_KEY']

  auth_config = weaviate.AuthApiKey(api_key=weaviate_api_key)

  client = weaviate.Client(
      url=weaviate_url,
      auth_client_secret=auth_config,
      additional_headers={
          "X-OpenAI-Api-Key": openai_api_key
      }
  )
  return Weaviate(client=client, 
                  index_name=index_name, 
                  text_key=text_key, 
                  attributes=attributes)


In [16]:
from langchain.agents.agent_toolkits import (
    create_vectorstore_agent,
    VectorStoreToolkit,
    VectorStoreInfo,
)

weaviate = weaviate_vector_store(index_name="Work", 
                                 text_key="title", 
                                 attributes=["identifier", "source", "title"])

vectorstore_info = VectorStoreInfo(
    name="Work",
    description="NULDC Works",
    vectorstore=weaviate,
)
llm = openai_chat_client()
toolkit = VectorStoreToolkit(vectorstore_info=vectorstore_info, llm=llm)
agent_executor = create_vectorstore_agent(llm=llm, toolkit=toolkit, verbose=True)


In [17]:
agent_executor.run("What can you tell me about John Cage's correspondence?")



> Entering new  chain...
I should use the Work tool to find information about John Cage's correspondence.
Action: Work
Action Input: John Cage's correspondence
Observation: I'm sorry, but I would need more information to answer your question. Which specific correspondence of John Cage are you referring to?
Thought:Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/ec2-user/.conda/envs/weaviate/lib/python3.9/site-packages/urllib3/connectionpool.py", line 377, in _make_request
    httplib_response = conn.getresponse(buffering=True)
TypeError: getresponse() got an unexpected keyword argument 'buffering'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ec2-user/.conda/envs/weaviate/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_6427/2344850719.py", line 1, in <module>
    agent_executor.run("What can you tell me about John Cage's correspondence?")
  File "/home/ec2-user/.conda/envs/weaviate/lib/python3.9/site-packages/langchain/chains/base.py", line 290, in run
    return self(args[0], callbacks=callbacks, tags=tags)[_output_key]
  File "/home/ec2-user/.conda/envs/weaviate/lib/python3.9/site-packages/langchain/chains/b